# Examine genes in Hetionet which do not have UMLS CUIs

2019-01-25

Here we will see if there are ways to map missed genes in Hetionet to UMLS CUIs.

In [1]:
import pandas as pd
import requests
from tqdm import tqdm
from collections import defaultdict

## Read Hetionet to UMLS mapping file

In [2]:
hnodes = pd.read_csv("../pipeline/hetionet/hetionet_nodes_umls.tsv", sep='\t')

In [3]:
hnodes.shape

(58700, 4)

In [4]:
hnodes.head()

,hetio_id,name,het_type,cui
0,1,A1BG,Gene,UMLS:C1412045
1,10,NAT2,Gene,UMLS:C0796518
2,100,ADA,Gene,UMLS:C1412179
3,1000,CDH2,Gene,UMLS:C1413277
4,10000,AKT3,Gene,UMLS:C1332074


## Select Hetionet genes with no CUI

In [5]:
missed_genes = (hnodes
    .assign(is_cui = lambda df: df["cui"].str.startswith("UMLS:C"))
    .query("~is_cui and het_type == 'Gene'")
    .reset_index(drop=True)
)

In [6]:
missed_genes.shape

(1562, 5)

In [7]:
missed_genes.head()

,hetio_id,name,het_type,cui,is_cui
0,100049579,OCTN3,Gene,100049579,False
1,100127889,C10orf131,Gene,100127889,False
2,100127971,LOC100127971,Gene,100127971,False
3,100127991,LOC100127991,Gene,100127991,False
4,100128001,LOC100128001,Gene,100128001,False


---

## Differentiate between LOC genes and named genes

In [8]:
missed_genes = missed_genes.assign(is_loc = lambda df: df["name"].str.startswith("LOC"))

In [9]:
missed_genes.head()

,hetio_id,name,het_type,cui,is_cui,is_loc
0,100049579,OCTN3,Gene,100049579,False,False
1,100127889,C10orf131,Gene,100127889,False,False
2,100127971,LOC100127971,Gene,100127971,False,True
3,100127991,LOC100127991,Gene,100127991,False,True
4,100128001,LOC100128001,Gene,100128001,False,True


In [10]:
missed_genes["is_loc"].value_counts()

True     1460
False     102
Name: is_loc, dtype: int64

## Functions for checking gene status

In [11]:
def make_clickable(val):
    return '<a href="https://www.ncbi.nlm.nih.gov/gene/{}">{}</a>'.format(val, val)

In [12]:
def parse(gene_id):
    """
    Determine gene status and HGNC id
    for Entrez gene ids using MyGene.info
    """

    def process(data):
        hgnc = data.get("HGNC")
        
        if data["_id"] != gene_id:
            return ("replaced", hgnc)
        
        return ("normal", hgnc)
    
    url = "http://mygene.info/v3/gene/{}".format(gene_id)
    resp = requests.get(url)
    
    if resp.status_code == 404:
        return ("withdrawn", None)
    
    assert resp.status_code == 200
    return process(resp.json())

---

## Get gene status for all missed genes

In [13]:
res = defaultdict(list)

for row in tqdm(missed_genes.itertuples(), total=len(missed_genes)):
    hetio_id = row.hetio_id
    status, hgnc_id = parse(hetio_id)
    
    res["hetio_id"].append(hetio_id)
    res["gene_status"].append(status)
    res["hgnc_id"].append(hgnc_id)
    
res = pd.DataFrame(res)

100%|██████████| 1562/1562 [02:21<00:00, 10.99it/s]


In [14]:
res.shape

(1562, 3)

In [15]:
res.head()

,hetio_id,gene_status,hgnc_id
0,100049579,normal,None
1,100127889,replaced,31666
2,100127971,withdrawn,None
3,100127991,withdrawn,None
4,100128001,withdrawn,None


---

## Merge gene status results with original data

In [16]:
final_res = (missed_genes
    [["hetio_id", "name", "is_loc"]]
    .merge(res, how="inner", on="hetio_id")
)

In [17]:
final_res.shape

(1562, 5)

In [18]:
final_res.head()

,hetio_id,name,is_loc,gene_status,hgnc_id
0,100049579,OCTN3,False,normal,None
1,100127889,C10orf131,False,replaced,31666
2,100127971,LOC100127971,True,withdrawn,None
3,100127991,LOC100127991,True,withdrawn,None
4,100128001,LOC100128001,True,withdrawn,None


---

# Results

## Gene status of all missed genes

In [19]:
final_res["gene_status"].value_counts(normalize=True).multiply(100)

withdrawn    48.65557
normal       40.78105
replaced     10.56338
Name: gene_status, dtype: float64

### Gene status divided by LOC status

In [20]:
final_res.groupby("is_loc")["gene_status"].value_counts()

is_loc  gene_status
False   withdrawn       41
        normal          31
        replaced        30
True    withdrawn      719
        normal         606
        replaced       135
Name: gene_status, dtype: int64

In [21]:
final_res.groupby("is_loc")["gene_status"].value_counts(normalize=True).multiply(100)

is_loc  gene_status
False   withdrawn      40.196078
        normal         30.392157
        replaced       29.411765
True    withdrawn      49.246575
        normal         41.506849
        replaced        9.246575
Name: gene_status, dtype: float64

## Do the non-withdrawn genes have HGNC ids?

In [22]:
final_res["hgnc_id"].notnull().sum()

171

In [23]:
final_res.groupby(["is_loc", "gene_status"]).apply(lambda df: df["hgnc_id"].notnull().sum())

is_loc  gene_status
False   normal          4
        replaced       30
        withdrawn       0
True    normal         43
        replaced       94
        withdrawn       0
dtype: int64

There are an additional 171 genes we can map to HGNC.

---

## Save results to file

In [24]:
final_res.to_csv("../pipeline/node_analysis_missed_genes.tsv", sep='\t', index=False)

# Conclusion

Most of the genes we could not map from Hetionet ids to UMLS CUIs are not mappable because the genes themselves are no longer considered to be genes. The vast majority of remaining genes do not have any cross references, and a very small subset of genes can be mapped to CUIs.

We should probably remove any edges involving withdrawn genes from Hetionet, and map the remaining genes.